__Prerequisits__

To run this sample either a [GPT-4 Turbo with Vision](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/gpt-with-vision?tabs=rest%2Csystem-assigned%2Cresource) model or [GPT-4o](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-4o-and-gpt-4-turbo) model is requiered.

In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs
#!import plugins/Math.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

__Adding plugins__

Compared to the other samples we are now also adding a plugin to the kernel.

SemanticKernel uses the [OpenAI Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant) to execute functions.

To do so functions musst have the [KernelFunction](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.kernelfunction?view=semantic-kernel-dotnet) attribute and the [Description](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.kernelfunction.description?view=semantic-kernel-dotnet#microsoft-semantickernel-kernelfunction-description) attribute. 

 - [KernelFunction](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.kernelfunction?view=semantic-kernel-dotnet): Lets the kernel know this is a function.
 - [Description](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.kernelfunction.description?view=semantic-kernel-dotnet#microsoft-semantickernel-kernelfunction-description) lets the kernel know what the function is about and creates the [Tools](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant-functions?tabs=python) used by Azure OpenAI.

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");
var builder = Kernel.CreateBuilder();

builder
    .AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey)
    .Plugins.AddFromType<MathPlugin>();

var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = 
"""
You're chatting with a user. 
You are an expert in mathematics from the 17th century.
The user will ask you to support them in solving a math problem.
Always be ver polite and talk in a formal way, the user is like a king.
""";

var chat = new ChatHistory(systemMessage);

__Streaming__

To use plugins we have to use a different GetMessage Method. 

Instead of using [GetChatMessageContentAsync](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.chatcompletion.ichatcompletionservice.getchatmessagecontentsasync?view=semantic-kernel-dotnet) we have to use [GetStreamingChatMessageContentsAsync](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.chatcompletion.ichatcompletionservice.getstreamingchatmessagecontentsasync?view=semantic-kernel-dotnet). If we would not change to the streaming method, semantic kernel is not able to call functions.

In [ ]:
var promptSettings = new OpenAIPromptExecutionSettings()
    {
        MaxTokens = 4000,
        ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions,
    };

Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User:");
    Console.WriteLine($"{input}\n");
    chat.AddUserMessage(input);

     StringBuilder sb = new();
    await foreach (var message in chatGPT.GetStreamingChatMessageContentsAsync(chat, promptSettings, kernel))
    {
        sb.Append(message.Content);
    }
    var assistantReply = sb.ToString();
    chat.AddAssistantMessage(assistantReply);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
await Chat("What is the sqaure root of 16?");

In [ ]:
//await Chat("Can you multiplie the reulst by 5");